# Week 4 homework

In this homework, we'll deploy the ride duration model in batch mode. Like in homework 1, we'll use the Yellow Taxi Trip Records dataset.

In [ ]:
!pip freeze | grep scikit-learn

In [ ]:
!python -V

In [ ]:
import pickle
import numpy as np
import pandas as pd

In [ ]:
with open("week4_model.bin", "rb") as f_in:
    dv, model = pickle.load(f_in)

In [ ]:
categorical = ["PULocationID", "DOLocationID"]


def read_data(filename):
    df = pd.read_parquet(filename)

    df["duration"] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df["duration"] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype("int").astype("str")

    return df

## Working with data from March, 2023

In [ ]:
taxi_type = "yellow"
year = 2023
month = 3

In [ ]:
df = read_data(
    f"https://d37ci6vzurychx.cloudfront.net/trip-data/{taxi_type}_tripdata_{year:04d}-{month:02d}.parquet"
)

df.head()

In [ ]:
dicts = df[categorical].to_dict(orient="records")
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

## Question 1

In [ ]:
np.std(y_pred)

## Question 2

In [ ]:
df["ride_id"] = f"{year:04d}/{month:02d}_" + df.index.astype("str")

In [ ]:
df_result = pd.DataFrame()
df_result["ride_id"] = df["ride_id"]
df_result["predicted_duration"] = y_pred

In [ ]:
output_file = f"output/{taxi_type}-{year:04d}-{month:02d}.parquet"

In [ ]:
df_result.to_parquet(output_file, engine="pyarrow", compression=None, index=False)